In [ ]:
forms = {
    "appeal_number": {"bench": "bench_name_1", "appeal": "app_type_1", "input": "app_number", "year": "app_year_1", "button": "b1"},
    "order_date": {"bench": "bench_name_2", "appeal": "app_type_2", "date": "order_date", "button": "b2"},
    "pron_date": {"bench": "bench_name_3", "appeal": "app_type_3", "date": "pron_date", "button": "b3"},
    "member_name": {"bench": "bench_name_4", "member": "member", "date": "order_date_2", "button": "b4"},
}


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import os
from pydub import AudioSegment
import speech_recognition as sr

driver = webdriver.Chrome()
driver.get("https://itat.gov.in/judicial/tribunalorders")

time.sleep(2)
audio_btn = driver.find_element(By.XPATH, "//img[@alt='Play Icon']")
audio_btn.click()
time.sleep(2)

audio_src = driver.find_element(By.ID, "captchaAudio").get_attribute("src")
print("Audio URL:", audio_src)

session = requests.Session()
for cookie in driver.get_cookies():
    session.cookies.set(cookie['name'], cookie['value'])

response = session.get(audio_src)
if response.status_code == 200:
    content_type = response.headers.get("Content-Type", "")
    ext = ".mp3" if "mpeg" in content_type or "mp3" in content_type else ".wav"
    mp3_filename = "captcha_audio.mp3"
    with open(mp3_filename, "wb") as f:
        f.write(response.content)
    print(f"Audio saved successfully as {mp3_filename}")
else:
    print("Failed to download audio:", response.status_code)
    driver.quit()
    exit()

wav_path = "captcha_audio.wav"
sound = AudioSegment.from_file(mp3_filename, format="mp3")
sound.export(wav_path, format="wav")
print(f"Converted {mp3_filename} to {wav_path}")

recognizer = sr.Recognizer()
with sr.AudioFile(wav_path) as source:
    audio_data = recognizer.record(source)

try:
    captcha_text = recognizer.recognize_google(audio_data).upper().replace(" ", "")
    print("Recognized Captcha:", captcha_text)
except sr.UnknownValueError:
    print("Could not understand audio")
    captcha_text = ""
except sr.RequestError as e:
    print("Recognition failed:", e)
    captcha_text = ""
    
if captcha_text:
    captcha_input = driver.find_element(By.ID, "captcha")
    captcha_input.clear()
    #human-like typing
    for char in captcha_text:
        captcha_input.send_keys(char)
        time.sleep(0.2)

time.sleep(10)
driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get("https://itat.gov.in/judicial/tribunalorders")

wait = WebDriverWait(driver, 10)
accordion_btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#headingTwo button")))
driver.execute_script("arguments[0].click();", accordion_btn)  # bypass overlay issues
time.sleep(1)

Select(driver.find_element(By.ID, "bench_name_2")).select_by_index(1)
time.sleep(2)
Select(driver.find_element(By.ID, "app_type_2")).select_by_index(1)
time.sleep(2)

order_date_input = driver.find_element(By.ID, "order_date")
driver.execute_script("arguments[0].removeAttribute('readonly')", order_date_input)
order_date_input.clear()
order_date_input.send_keys("11/09/2025")
order_date_input.send_keys(Keys.TAB)

driver.find_element(By.ID, "b2").click()
time.sleep(5)

driver.quit()


The chromedriver version (139.0.7258.138) detected in PATH at C:\Users\rando\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (140.0.7339.81); currently, chromedriver 140.0.7339.82 is recommended for chrome 140.*, so it is advised to delete the driver in PATH and retry


In [24]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests, time, os
from pydub import AudioSegment
import speech_recognition as sr

driver = webdriver.Chrome()
driver.get("https://itat.gov.in/judicial/tribunalorders")

wait = WebDriverWait(driver, 10)

accordion_btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#headingTwo button")))
driver.execute_script("arguments[0].click();", accordion_btn)
time.sleep(1)


Select(driver.find_element(By.ID, "bench_name_2")).select_by_index(1)
time.sleep(1)
Select(driver.find_element(By.ID, "app_type_2")).select_by_index(1)
time.sleep(1)

order_date_input = driver.find_element(By.ID, "order_date")
driver.execute_script("arguments[0].removeAttribute('readonly')", order_date_input)
order_date_input.clear()
order_date_input.send_keys("15/08/2025")
order_date_input.send_keys(Keys.TAB)
time.sleep(1)


driver.find_element(By.XPATH, "//img[@alt='Play Icon']").click()
time.sleep(2)
audio_src = driver.find_element(By.ID, "captchaAudio").get_attribute("src")


session = requests.Session()
for cookie in driver.get_cookies():
    session.cookies.set(cookie['name'], cookie['value'])
response = session.get(audio_src)
filename_mp3 = "captcha_audio.mp3"
with open(filename_mp3, "wb") as f:
    f.write(response.content)


wav_path = "captcha_audio.wav"
AudioSegment.from_file(filename_mp3).export(wav_path, format="wav")


recognizer = sr.Recognizer()
with sr.AudioFile(wav_path) as source:
    audio_data = recognizer.record(source)
try:
    captcha_text = recognizer.recognize_google(audio_data).upper().replace(" ", "")
except:
    captcha_text = ""
print("Captcha:", captcha_text)


captcha_input = driver.find_element(By.ID, "captcha")
for char in captcha_text:
    captcha_input.send_keys(char)
    time.sleep(0.2)  # imitate human typing

driver.find_element(By.ID, "b2").click()
time.sleep(5)

driver.quit()


The chromedriver version (139.0.7258.138) detected in PATH at C:\Users\rando\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (140.0.7339.81); currently, chromedriver 140.0.7339.82 is recommended for chrome 140.*, so it is advised to delete the driver in PATH and retry


Captcha: M59J6


APPEAL OPTIONS

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://itat.gov.in/judicial/tribunalorders")

wait = WebDriverWait(driver, 10)
accordion_btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#headingTwo button")))
driver.execute_script("arguments[0].click();", accordion_btn)


wait.until(EC.presence_of_element_located((By.ID, "bench_name_2")))


bench_select = Select(driver.find_element(By.ID, "bench_name_2"))
bench_options = [opt.get_attribute("value") for opt in bench_select.options if opt.get_attribute("value")]


appeal_select = Select(driver.find_element(By.ID, "app_type_2"))
appeal_options = [opt.get_attribute("value") for opt in appeal_select.options if opt.get_attribute("value")]

print("Bench Options:", bench_options)
print("Appeal Options:", appeal_options)

driver.quit()


In [6]:
from itertools import product

benches = ['203', '205', '207', '209', '211', '215', '217', '219', '221', '260', '201', '223', '225', '227', '229', '231', '233', '235', '237', '199', '239', '241', '243', '245', '247', '249', '251', '256', '258', '253']
appeal_types = ['ITA', 'CO', 'ITSSA', 'ITTPA', 'ITITA', 'WTA', 'BMA', 'EDA', 'INTTA', 'GTA', 'TDS', 'STTA', 'ETA', 'STA', 'HCD', 'SA', 'MA', 'RA']

combinations = list(product(benches, appeal_types))

print(f"Total combinations: {len(combinations)}")
print(combinations[:10])  # show first 10 for sanity check


Total combinations: 540
[('203', 'ITA'), ('203', 'CO'), ('203', 'ITSSA'), ('203', 'ITTPA'), ('203', 'ITITA'), ('203', 'WTA'), ('203', 'BMA'), ('203', 'EDA'), ('203', 'INTTA'), ('203', 'GTA')]
